# Hyperparameters tuning for CWT keras v1

Check the name for val_kullback_leibler_divergence.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import regularizers, layers
# import sys

import keras_tuner as kt

# ----------------------------------------
# Flags for working on different hardware.
# flag_kaggle = True
flag_FW = True
# flag_LN = True

try:
    if flag_kaggle:
        # sys.path.insert(0, '/kaggle/input/hms-lib')
        base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'
        devset_dir = '/kaggle/input/hms-cwt-scalograms-single-numpy-v1'
except:
    pass 

try:
    if flag_FW:
        # sys.path.insert(0, '../lib')
        base_dir = '../../kaggle_data/hms'
        devset_dir = '../data'
except:
    pass 

try:
    if flag_LN:
        # sys.path.insert(0, '../lib')
        base_dir = '../../data/hms'
        devset_dir = '../data'
except:
    pass 
# ----------------------------------------

# path_train = f'{devset_dir}/05_single_cwt_v1_train.npy'
# path_train_items = f'{devset_dir}/05_single_cwt_v1_train_items.npy'
# path_val = f'{devset_dir}/05_single_cwt_v1_val.npy'
# path_val_items = f'{devset_dir}/05_single_cwt_v1_val_items.npy'
# path_test = f'{devset_dir}/05_single_cwt_v1_test.npy'
# path_test_items = f'{devset_dir}/05_single_cwt_v1_test_items.npy'


In [3]:
path_train = f'{devset_dir}/05_reduced_single_cwt_v1_train.npy'
path_train_items = f'{devset_dir}/05_reduced_single_cwt_v1_train_items.npy'
path_val = f'{devset_dir}/05_reduced_single_cwt_v1_val.npy'
path_val_items = f'{devset_dir}/05_reduced_single_cwt_v1_val_items.npy'
path_test = f'{devset_dir}/05_reduced_single_cwt_v1_test.npy'
path_test_items = f'{devset_dir}/05_reduced_single_cwt_v1_test_items.npy'

The my_dir/intro_to_kt directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.

In [5]:
#
# Data generator.
#
# cwt's coefficients
# 5 channels (LP, RP, LT, RP, C)
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y

In [6]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

# Dimensions for input shape.
dim = training_generator.get_dim()

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 512
Observations in validation set: 128


In [15]:
def model_builder(hp):
    model = keras.Sequential()

    model.add(layers.Conv2D(
        # filters=hp.Int('conv_1_filter', min_value=32, max_value=64, step=16),
        filters=hp.Choice('conv_1_filter', values = [32, 64]),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        padding="same",
        activation='relu',
        input_shape=dim)
    ),


    # model.add(layers.Flatten())
    # # Tune the number of layers.
    # for i in range(hp.Int("num_layers", 1, 3)):
    #     model.add(
    #         layers.Dense(
    #             # Tune number of units separately.
    #             units=hp.Int(f"units_{i}", min_value=32, max_value=64, step=32),
    #             # activation=hp.Choice("activation", ["relu", "tanh"]),
    #             activation="relu",
    #         )
    #     )
    # if hp.Boolean("dropout"):
        # model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    # learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    learning_rate = 0.001
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.KLDivergence(),
        # metrics=["accuracy"],
        # Objective is one of the metrics.
        metrics=[tf.keras.metrics.KLDivergence()],
    )
    return model

# model_builder(kt.HyperParameters())


In [ ]:
# tuner = kt.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="my_dir",
#     project_name="helloworld",
# )

In [16]:
tuner = kt.Hyperband(model_builder,
                    #  objective='val_accuracy',
                     objective=kt.Objective("val_kullback_leibler_divergence", direction="min"),
                     max_epochs=10,
                     factor=3,
                     directory='results',
                     project_name='cwt_v1')


In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_kullback_leibler_divergence', patience=5)

In [17]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}


In [18]:
tuner.search(training_generator, epochs=2, validation_data=validation_generator, callbacks=[stop_early])

Trial 2 Complete [00h 00m 01s]

Best val_kullback_leibler_divergence So Far: None
Total elapsed time: 00h 00m 01s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
64                |32                |units_0
32                |32                |units_1
64                |32                |units_2
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


Traceback (most recent call last):
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/edg/miniconda3/envs/gsdc/lib/python3.10/site-packages/keras_tuner/src/engine/tune

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()


In [ ]:
# tuner.results_summary()

In [ ]:
# Retrain the model

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)


In [ ]:
# Name to monitor is different.
checkpoint_filepath = 'results/checkpoint1.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_kullback_leibler_divergence',
    mode='min',
    save_best_only=True)


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))
